In [1]:
from sklearn.cluster import KMeans
import torch
from torch.nn import Embedding
from torch.utils.data import DataLoader
import torch.nn as nn
EPS = 1e-15
from torch.nn import Parameter
from torch_sparse import SparseTensor
from torch_geometric.utils.num_nodes import maybe_num_nodes
import pandas as pd
import numpy as np
import torch_cluster  
random_walk = torch.ops.torch_cluster.random_walk
device = 'cpu'
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, average_precision_score
from sklearn import metrics
from torch_geometric.datasets import Planetoid
beta = 4.5
gama = 0.5

def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    
    print("预测精确率： ",average_precision_score(true_edges, predict_edges))
    fpr, tpr, _ = metrics.roc_curve(true_edges, predict_edges, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    predict_edges[predict_edges>gama] = 1
    predict_edges[predict_edges<=gama] = 0
    
    print("预测准确率： ",accuracy_score(true_edges, predict_edges))
    #print("预测精确率： ",precision_score(true_edges, predict_edges, average='macro'))
    #print("召回率： ",recall_score(true_edges, predict_edges, average='macro'))
    print("F1 SCORE： ",f1_score(true_edges, predict_edges, average='macro'))
    
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, ne, alpha=1.0):
        super(reconstruction_graph, self).__init__()
        NE = ne
        self.alpha = alpha
        self.nodes_embedding = Parameter(NE)
        
    def forward(self, cc):
        CC = cc
        # 计算每个节点对每个类簇的注意力分数（以节点为单位）
        norm_squared = torch.sum((self.nodes_embedding.unsqueeze(1) - CC)**2, 2)   # 节点到质心的距离
        numerator = 1.0 / (1.0 + (norm_squared / self.alpha))
        power = float(self.alpha + 1) / 2    # 计算幂
        numerator = numerator**power    # 所有节点
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() #soft assignment using t-distribution
        # torch.sum(numerator, 1) : 对每一行进行相加
        
        # 计算节点之间的类簇相似性（余弦相似性）：值越大越有利于边的形成
        prod = torch.mm(soft_assignments, soft_assignments.t())#分子
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)#分母
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.nodes_embedding[:, None]-self.nodes_embedding, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # 计算边的形成概率
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

class update_nodes_embedding(nn.Module):
    def __init__(self, ne):
        super(update_nodes_embedding, self).__init__()
        NE = ne
        self.reconstruction_module = reconstruction_graph(NE)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=0.4, momentum=0.9)
        #self.loss_function = torch.nn.L1Loss(reduction='sum')
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, g, cc, edge_train, edge_test):
        CC = cc
        GP = g
        self.reconstruction_module.train()
        for epoch in range(5):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(CC)
            graph_train = torch.take(graph_reconstruction, edge_train)
            loss = self.loss_function(g, graph_train)
            loss.backward()
            self.optimizer.step()
            #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test).detach()
        predict_acc(recons_test_edges, test_edge)
        return self.reconstruction_module.nodes_embedding.detach()
    
####################################### 以最大化预测效果为目标的类型质心更新 ########################################
class ClusteringLayer(nn.Module):
    """给定节点嵌入，更新类簇质心"""
    def __init__(self, cc, alpha=1.0):
        super(ClusteringLayer, self).__init__()
        self.alpha = alpha
        self.cluster_centers = Parameter(cc)
    
    def forward(self, x):
        NE = x
        # 计算每个节点对每个类簇的注意力分数（以节点为单位）
        norm_squared = torch.sum((NE.unsqueeze(1) - self.cluster_centers)**2, 2)   # 节点到质心的距离
        numerator = 1.0 / (1.0 + (norm_squared / self.alpha))
        power = float(self.alpha + 1) / 2    # 计算幂
        numerator = numerator**power    # 所有节点
        soft_assignments = (numerator.t() / torch.sum(numerator, 1)).t() #soft assignment using t-distribution
        # torch.sum(numerator, 1) : 对每一行进行相加
        
        # 计算节点之间的类簇相似性（余弦相似性）：值越大越有利于边的形成
        prod = torch.mm(soft_assignments, soft_assignments.t())#分子
        norm = torch.norm(soft_assignments,p=2,dim=1).unsqueeze(0)#分母
        clusters_similar = prod.div(torch.mm(norm.t(),norm))
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(NE[:, None]-NE, dim=2, p=2)
        nodes_distance = torch.div(nodes_distance, torch.max(nodes_distance))
        
        # 计算边的形成概率
        distance_similar = torch.div(beta*nodes_distance, clusters_similar)      
        nodes_similar = torch.exp(-distance_similar)
        
        return nodes_similar

def find_cluster_centers(ne, ini_cc, g, edge_train, edge_test, num_epochs=5):        
    NE = ne
    CC = ini_cc
    clusteringlayer = ClusteringLayer(CC)
    optimizer = torch.optim.SGD(params=clusteringlayer.parameters(), lr=0.4, momentum=0.9)
    #loss_function = torch.nn.L1Loss(reduction='sum')
    loss_function = torch.nn.MSELoss(reduction='sum')
    
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        graph_recons = clusteringlayer(NE)
        graph_train = torch.take(graph_recons, edge_train)
        loss = loss_function(g, graph_train)
        loss.backward()
        optimizer.step()
        #print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')     
    recons_test_edges = torch.take(graph_recons, edge_test).detach()
    predict_acc(recons_test_edges, test_edge)
    return clusteringlayer.cluster_centers.detach()

In [2]:
def get_cora_graph(filename_adj, nodes_number):
    raw_edges = pd.read_csv(filename_adj, header=None, sep='\t')
    
    drop_self_loop = raw_edges[raw_edges[0]!=raw_edges[1]]    # 去掉自循环
    
    graph_np = np.zeros((nodes_number,nodes_number))
    for i in range(drop_self_loop.shape[0]):
        graph_np[drop_self_loop.iloc[i,0],drop_self_loop.iloc[i,1]]=1
        graph_np[drop_self_loop.iloc[i,1],drop_self_loop.iloc[i,0]]=1

    posi_edge = np.argwhere(graph_np == 1)
    posi_edge = posi_edge[posi_edge[:,0]<posi_edge[:,1]]         # 只取左上角矩阵
    posi_edge_number = posi_edge.shape[0]
    
    nega_edge = np.argwhere(graph_np == 0) 
    nega_edge = nega_edge[nega_edge[:,0]<nega_edge[:,1]]     # 只取左上角矩阵
    
    positive_index = np.random.choice(range(posi_edge_number),int(posi_edge_number*0.9),replace=False)
    choose_positive = posi_edge[positive_index]
    posi_not_choose = np.setdiff1d(range(posi_edge_number), positive_index)
    test_positive = posi_edge[posi_not_choose]
    
    negative_index = np.random.choice(range(nega_edge.shape[0]),int(posi_edge_number*0.9)*4,replace=False)
    choose_negative = nega_edge[negative_index]
    nega_not_choose = np.setdiff1d(range(nega_edge.shape[0]), negative_index)
    test_nega_index = np.random.choice(nega_not_choose,len(posi_not_choose),replace=False)
    test_negative = nega_edge[test_nega_index]
    
    train_posi = [list(choose_positive[i]) for i in range(len(choose_positive))]
    train_nega = [list(choose_negative[i]) for i in range(len(choose_negative))]
    train_index = train_posi + train_nega
    train_mask = [train_index[i][0]*nodes_number+train_index[i][1] for i in range(len(train_index))]
    train_mask = torch.tensor(train_mask)
    
    test_posi = [list(test_positive[i]) for i in range(len(test_positive))]
    test_nega = [list(test_negative[i]) for i in range(len(test_negative))]
    test_index = test_posi + test_nega
    test_mask = [test_index[i][0]*nodes_number+test_index[i][1] for i in range(len(test_index))]
    test_mask = torch.tensor(test_mask)
    
    graph_tensor = torch.from_numpy(graph_np).float()
    train_edge = torch.take(graph_tensor, train_mask)
    test_edge = np.array(torch.take(graph_tensor, test_mask))
    
    return train_edge, test_edge, train_mask, test_mask

In [3]:
filename = "data/graph.txt"
nodes_number = 2405
train_edge, test_edge, train_mask, test_mask = get_cora_graph(filename, nodes_number)

In [4]:
embedding_dim = 10
n_clusters = 24
alpha = 1

ini_embedding = Embedding(nodes_number, embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
ini_node_embeddings = ini_embedding.weight.detach()

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(ini_node_embeddings)
cluster_centers = kmeans.cluster_centers_
cluster_centers = torch.tensor(cluster_centers, dtype=torch.float)

In [5]:
raw_nodes_embedding = ini_node_embeddings     # t=0时刻的节点嵌入
raw_cluster_centers = cluster_centers     # t=0时刻的类簇质心
for module_epoch in range(500):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(raw_nodes_embedding)
    raw_nodes_embedding = update_nodes_module(train_edge, raw_cluster_centers, train_mask, test_mask)
    raw_cluster_centers = find_cluster_centers(raw_nodes_embedding, raw_cluster_centers, train_edge, train_mask, test_mask)

######################### 模块循环 ： 0 ##########################
预测精确率：  0.6323511389236975
AUC SCORE:  0.6011957491082045
预测准确率：  0.4995689655172414
F1 SCORE：  0.33466637848519465
预测精确率：  0.6401395030243352
AUC SCORE:  0.6026222502972652
预测准确率：  0.47672413793103446
F1 SCORE：  0.32497363627648357
######################### 模块循环 ： 1 ##########################
预测精确率：  0.6882453044690775
AUC SCORE:  0.669460835315101
预测准确率：  0.5008620689655172
F1 SCORE：  0.33751814910674827
预测精确率：  0.684439836719488
AUC SCORE:  0.6612254756242568
预测准确率：  0.6094827586206897
F1 SCORE：  0.609453734671126
######################### 模块循环 ： 2 ##########################
预测精确率：  0.7360404291597147
AUC SCORE:  0.7064952437574317
预测准确率：  0.6331896551724138
F1 SCORE：  0.6322543716732005
预测精确率：  0.7333414352425245
AUC SCORE:  0.7026129607609988
预测准确率：  0.5831896551724138
F1 SCORE：  0.5052136460146193
######################### 模块循环 ： 3 ##########################
预测精确率：  0.7611664769100916
AUC SCORE:  0.7289967300832343
预测准

预测精确率：  0.8898919326536435
AUC SCORE:  0.864502824019025
预测准确率：  0.7392241379310345
F1 SCORE：  0.7226586280612324
预测精确率：  0.8903486078482743
AUC SCORE:  0.864932372175981
预测准确率：  0.7392241379310345
F1 SCORE：  0.7226586280612324
######################### 模块循环 ： 29 ##########################
预测精确率：  0.8913968517698243
AUC SCORE:  0.8661957491082045
预测准确率：  0.7439655172413793
F1 SCORE：  0.7283642246884845
预测精确率：  0.891842483770517
AUC SCORE:  0.8665651010701546
预测准确率：  0.7452586206896552
F1 SCORE：  0.7299132429616646
######################### 模块循环 ： 30 ##########################
预测精确率：  0.8927443387423446
AUC SCORE:  0.8676657253269917
预测准确率：  0.7469827586206896
F1 SCORE：  0.7319739681113892
预测精确率：  0.8930873872374361
AUC SCORE:  0.86785225921522
预测准确率：  0.7478448275862069
F1 SCORE：  0.7330023601316574
######################### 模块循环 ： 31 ##########################
预测精确率：  0.8941369405616335
AUC SCORE:  0.8692516349583829
预测准确率：  0.7491379310344828
F1 SCORE：  0.7345425033972519
预测精确率：  0.8

预测精确率：  0.9110190826647173
AUC SCORE:  0.8947228002378121
预测准确率：  0.7754310344827586
F1 SCORE：  0.7662827388727037
######################### 模块循环 ： 57 ##########################
预测精确率：  0.9134852282950183
AUC SCORE:  0.897343192627824
预测准确率：  0.7758620689655172
F1 SCORE：  0.766855079096115
预测精确率：  0.9150809372969639
AUC SCORE:  0.8986831153388823
预测准确率：  0.7732758620689655
F1 SCORE：  0.7633995149798631
######################### 模块循环 ： 58 ##########################
预测精确率：  0.9161324500727415
AUC SCORE:  0.899080707491082
预测准确率：  0.7711206896551724
F1 SCORE：  0.7609295940448972
预测精确率：  0.9172225840076115
AUC SCORE:  0.900779577883472
预测准确率：  0.7728448275862069
F1 SCORE：  0.7629060798658076
######################### 模块循环 ： 59 ##########################
预测精确率：  0.9170027472362345
AUC SCORE:  0.9003344233055887
预测准确率：  0.7741379310344828
F1 SCORE：  0.7645571424587325
预测精确率：  0.9173848431482036
AUC SCORE:  0.9008226813317479
预测准确率：  0.7741379310344828
F1 SCORE：  0.7643855666778303
##########

预测精确率：  0.9190267926987563
AUC SCORE:  0.904092598097503
预测准确率：  0.7827586206896552
F1 SCORE：  0.7752518729010591
预测精确率：  0.9218243217297264
AUC SCORE:  0.907563168846611
预测准确率：  0.7814655172413794
F1 SCORE：  0.7735027223230491
######################### 模块循环 ： 86 ##########################
预测精确率：  0.9209067557439667
AUC SCORE:  0.9058524078478003
预测准确率：  0.784051724137931
F1 SCORE：  0.7764799749849569
预测精确率：  0.9204370679876966
AUC SCORE:  0.9053745541022592
预测准确率：  0.784051724137931
F1 SCORE：  0.7764799749849569
######################### 模块循环 ： 87 ##########################
预测精确率：  0.9233498019765446
AUC SCORE:  0.9102296373365042
预测准确率：  0.7879310344827586
F1 SCORE：  0.7806743479834813
预测精确率：  0.9236054407540211
AUC SCORE:  0.9108984839476815
预测准确率：  0.7866379310344828
F1 SCORE：  0.7792292054154591
######################### 模块循环 ： 88 ##########################
预测精确率：  0.9240121166304839
AUC SCORE:  0.9112641200951248
预测准确率：  0.7870689655172414
F1 SCORE：  0.7795667654507911
预测精确率：  0.

预测精确率：  0.9229933072863189
AUC SCORE:  0.9110813020214032
预测准确率：  0.7952586206896551
F1 SCORE：  0.7888231489687882
######################### 模块循环 ： 114 ##########################
预测精确率：  0.9227868644956887
AUC SCORE:  0.9101649821640905
预测准确率：  0.7965517241379311
F1 SCORE：  0.7902542064466798
预测精确率：  0.9230533090444598
AUC SCORE:  0.9108152497027349
预测准确率：  0.7943965517241379
F1 SCORE：  0.7878015822841242
######################### 模块循环 ： 115 ##########################
预测精确率：  0.9227978230583437
AUC SCORE:  0.9103195600475623
预测准确率：  0.7974137931034483
F1 SCORE：  0.7912071078431373
预测精确率：  0.9235000748123107
AUC SCORE:  0.9117717003567182
预测准确率：  0.7961206896551725
F1 SCORE：  0.789646851076465
######################### 模块循环 ： 116 ##########################
预测精确率：  0.9229235545932717
AUC SCORE:  0.9102749702734839
预测准确率：  0.7965517241379311
F1 SCORE：  0.7903186274509804
预测精确率：  0.9233111708700837
AUC SCORE:  0.91115338882283
预测准确率：  0.7956896551724137
F1 SCORE：  0.7892349763222837
######

...

预测精确率：  0.9294973088048057
AUC SCORE:  0.9261734542211654
预测准确率：  0.8181034482758621
F1 SCORE：  0.8139165711262017
预测精确率：  0.9294978403798961
AUC SCORE:  0.9263644470868014
预测准确率：  0.8168103448275862
F1 SCORE：  0.8124182540248746
######################### 模块循环 ： 370 ##########################
预测精确率：  0.9294744544261291
AUC SCORE:  0.926165279429251
预测准确率：  0.8181034482758621
F1 SCORE：  0.8139165711262017
预测精确率：  0.9294998912101337
AUC SCORE:  0.9263577586206897
预测准确率：  0.8168103448275862
F1 SCORE：  0.8124182540248746
######################### 模块循环 ： 371 ##########################
预测精确率：  0.9294518503013967
AUC SCORE:  0.9261912901307967
预测准确率：  0.8185344827586207
F1 SCORE：  0.8143820466435614
预测精确率：  0.9295306330459334
AUC SCORE:  0.9263867419738405
预测准确率：  0.8168103448275862
F1 SCORE：  0.8124182540248746
######################### 模块循环 ： 372 ##########################
预测精确率：  0.9294145388687172
AUC SCORE:  0.9262165576694411
预测准确率：  0.8181034482758621
F1 SCORE：  0.8139656494670782
预测精确

预测精确率：  0.9253781147335408
AUC SCORE:  0.9199004161712246
预测准确率：  0.8107758620689656
F1 SCORE：  0.8058099523734075
######################### 模块循环 ： 398 ##########################
预测精确率：  0.9286180470065342
AUC SCORE:  0.9238480975029726
预测准确率：  0.8271551724137931
F1 SCORE：  0.8236953442206651
预测精确率：  0.9216141279232433
AUC SCORE:  0.9153552318668252
预测准确率：  0.8224137931034483
F1 SCORE：  0.8187010448816578
######################### 模块循环 ： 399 ##########################
预测精确率：  0.924506479971031
AUC SCORE:  0.9189053210463733
预测准确率：  0.8103448275862069
F1 SCORE：  0.8053951005623868
预测精确率：  0.9259488089176959
AUC SCORE:  0.9207112068965517
预测准确率：  0.8150862068965518
F1 SCORE：  0.8106527787686382
######################### 模块循环 ： 400 ##########################
预测精确率：  0.9291704196006331
AUC SCORE:  0.9246269322235432
预测准确率：  0.825
F1 SCORE：  0.8214307380579722
预测精确率：  0.9297491235008981
AUC SCORE:  0.925477853745541
预测准确率：  0.8232758620689655
F1 SCORE：  0.8194896521570152
##################

预测精确率：  0.9223908942995989
AUC SCORE:  0.9170221462544591
预测准确率：  0.8150862068965518
F1 SCORE：  0.8108548646320376
######################### 模块循环 ： 426 ##########################
预测精确率：  0.9230728136137395
AUC SCORE:  0.9173082639714625
预测准确率：  0.8133620689655172
F1 SCORE：  0.808835515307374
预测精确率：  0.9235187224316216
AUC SCORE:  0.9177563912009512
预测准确率：  0.8146551724137931
F1 SCORE：  0.8102879606640377
######################### 模块循环 ： 427 ##########################
预测精确率：  0.9233345479501384
AUC SCORE:  0.9174123067776455
预测准确率：  0.8150862068965518
F1 SCORE：  0.8107037727441049
预测精确率：  0.923608235065605
AUC SCORE:  0.9175163495838289
预测准确率：  0.8112068965517242
F1 SCORE：  0.8068100153995323
######################### 模块循环 ： 428 ##########################
预测精确率：  0.923672418522969
AUC SCORE:  0.9177363258026161
预测准确率：  0.8116379310344828
F1 SCORE：  0.8070169728138417
预测精确率：  0.923669410801837
AUC SCORE:  0.9174925683709869
预测准确率：  0.8150862068965518
F1 SCORE：  0.8107544513907916
#######

预测精确率：  0.9284042779531583
AUC SCORE:  0.9228723246135553
预测准确率：  0.8103448275862069
F1 SCORE：  0.8056655498020103
######################### 模块循环 ： 454 ##########################
预测精确率：  0.9283542624279157
AUC SCORE:  0.9226553210463733
预测准确率：  0.8112068965517242
F1 SCORE：  0.8066543066680062
预测精确率：  0.928777450960684
AUC SCORE:  0.923002378121284
预测准确率：  0.8094827586206896
F1 SCORE：  0.8048355650564611
######################### 模块循环 ： 455 ##########################
预测精确率：  0.9288058310291065
AUC SCORE:  0.9231807372175982
预测准确率：  0.8103448275862069
F1 SCORE：  0.805771449620828
预测精确率：  0.9290308051438796
AUC SCORE:  0.923356866825208
预测准确率：  0.8103448275862069
F1 SCORE：  0.805771449620828
######################### 模块循环 ： 456 ##########################
预测精确率：  0.9287442067695236
AUC SCORE:  0.9230804102259215
预测准确率：  0.8120689655172414
F1 SCORE：  0.8075371637151842
预测精确率：  0.9289913512751146
AUC SCORE:  0.9232951843043995
预测准确率：  0.8107758620689656
F1 SCORE：  0.8061337552981154
########

预测精确率：  0.9283528101000242
AUC SCORE:  0.924049494649227
预测准确率：  0.8137931034482758
F1 SCORE：  0.8094564535447442
预测精确率：  0.9282493663906325
AUC SCORE:  0.9238711355529131
预测准确率：  0.8103448275862069
F1 SCORE：  0.805612112376308
######################### 模块循环 ： 483 ##########################
预测精确率：  0.928565612713312
AUC SCORE:  0.9243906064209275
预测准确率：  0.8112068965517242
F1 SCORE：  0.8066543066680062
预测精确率：  0.928653375612685
AUC SCORE:  0.9243958085612367
预测准确率：  0.8099137931034482
F1 SCORE：  0.8052505377823893
######################### 模块循环 ： 484 ##########################
预测精确率：  0.9287386609517507
AUC SCORE:  0.9245288347205708
预测准确率：  0.8103448275862069
F1 SCORE：  0.8057186620471558
预测精确率：  0.9288667529852355
AUC SCORE:  0.9245481569560048
预测准确率：  0.8099137931034482
F1 SCORE：  0.8052505377823893
######################### 模块循环 ： 485 ##########################
预测精确率：  0.9289607471199761
AUC SCORE:  0.9246863852556481
预测准确率：  0.8099137931034482
F1 SCORE：  0.8052505377823893
预测精确率： 